In [2]:
!pip -q install -U openai



In [32]:
import os
from openai import OpenAI

# This automatically reads from Colab Secrets / environment
client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)

print("Groq client initialized using secret key.")


Groq client initialized using secret key.


In [33]:
ROOT_PROMPT = """
You are The Structured Academic Tutor.

Role:
You act as a university instructor writing exam- and assignment-ready answers.

STRICT OUTPUT RULES (MANDATORY):
- Use ONLY plain text.
- DO NOT use Markdown.
- DO NOT use *, **, _, bullet symbols, emojis, or decorative formatting.
- Headings must be plain text followed by a colon.
- No conversational fillers such as greetings or casual phrases.

CORE BEHAVIOR (ROOT RULES):
1. Be accurate. If uncertain, say what is missing and how to verify.
2. Be structured. Always follow the required sections.
3. Be helpful. Provide clear explanations and practical examples when useful.
4. Use the patterns below whenever they improve quality.

PATTERN 1: Question Refinement Pattern
- If the user question is vague or underspecified, create a refined version of the question.
- Do not ask many questions. Ask at most 2 clarifying questions only if required.
- If you can proceed safely with reasonable assumptions, proceed and state assumptions.

PATTERN 2: Cognitive Verifier Pattern
Before finalizing the answer:
- List 3 to 5 quick checks you will use to verify correctness and completeness.
After the answer:
- Provide a short Verification section stating whether those checks passed and why.

PATTERN 3: Provide New Information and Ask Questions
- Add 2 to 4 helpful extra facts, tips, or context that the user did not explicitly ask for,
  but that increases understanding.
- Then ask 1 relevant follow-up question that helps personalize or deepen the next step.
- If the user explicitly wants no follow-up questions, skip the question.

REQUIRED STRUCTURE (follow exactly):
Plan:
- Use numbered points only (1., 2., 3., ...), 3 to 5 items.

Refined Question:
- Write a refined version of the user’s question in one sentence.

Verifier Checks:
- List 3 to 5 checks as numbered points.

Main Answer:
- Clear paragraphs.
- High-level step-by-step explanation.

New Information:
- 2 to 4 additional helpful points (numbered).

Follow-up Question:
- Ask exactly 1 question (if appropriate).

Summary:
- 3 to 4 concise sentences.

Verification:
- Briefly state if the verifier checks were satisfied.
"""


In [34]:
conversation_history = []

def ask_persona_ai(
    user_question,
    model="llama-3.1-8b-instant",
    temperature=0.25
):
    messages = [{"role": "system", "content": ROOT_PROMPT}]

    messages.extend(conversation_history)
    messages.append({"role": "user", "content": user_question})

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )

    answer = response.choices[0].message.content.strip()

    # Final safety cleanup
    answer = answer.replace("*", "").replace("_", "")

    conversation_history.append({"role": "user", "content": user_question})
    conversation_history.append({"role": "assistant", "content": answer})

    return answer


In [35]:
def make_chain_of_thought_practice_prompt(task, persona_name="Structured Academic Tutor"):
    """
    Safe chain-of-thought practice: uses a reasoning OUTLINE format (not hidden internal CoT).
    """
    return f"""
Role: {persona_name}
Task: {task}

Use this exact reasoning outline (plain text only):
Plan:
Key assumptions:
Steps:
Result:
Quick check:

Now produce the final output using that outline.
""".strip()


def make_tabular_prompt(topic, columns, rows_hint=None):
    """
    Forces tabular response using plain text (no Markdown tables).
    """
    cols = ", ".join(columns)
    hint = f"\nRow guidance: {rows_hint}" if rows_hint else ""
    return f"""
Task: Create a plain-text table (NOT Markdown).
Topic: {topic}

Rules:
- Output must be plain text only.
- Table must use columns: {cols}.
- Use a simple delimiter like: | between columns.
- One row per line.
- Keep each cell concise.{hint}

Return the table only.
""".strip()


def make_fill_in_blank_prompt(topic, blanks):
    """
    Creates a fill-in-the-blank worksheet prompt.
    blanks: list of (label, blank_description)
    """
    lines = [f"Topic: {topic}", "Fill in the blanks accurately:"]
    for i, (label, desc) in enumerate(blanks, 1):
        lines.append(f"{i}. {label}: ________ ({desc})")
    lines.append("Rules: plain text only. Do not add extra sections.")
    return "\n".join(lines)


def make_rgc_prompt(role, goal, constraints_must, constraints_avoid, format_rules, length=None):
    """
    Builds an RGC prompt block.
    """
    must = "\n".join([f"- {x}" for x in constraints_must])
    avoid = "\n".join([f"- {x}" for x in constraints_avoid])
    fmt = "\n".join([f"- {x}" for x in format_rules])
    length_line = f"\nLength: {length}" if length else ""
    return f"""
Role:
{role}

Goal:
{goal}

Constraints:
Must include:
{must}

Must avoid:
{avoid}

Format rules:
{fmt}{length_line}
""".strip()


def make_zero_shot_prompt(task, constraints=None):
    """
    Zero-shot: instructions only.
    """
    c = f"\nConstraints: {constraints}" if constraints else ""
    return f"""
Task: {task}{c}
Return the answer now.
""".strip()


def make_one_shot_prompt(task, example_q, example_a, constraints=None):
    """
    One-shot: includes one example.
    """
    c = f"\nConstraints: {constraints}" if constraints else ""
    return f"""
Task: {task}{c}

Example:
Question: {example_q}
Answer:
{example_a}

Now answer this:
Question: {task}
""".strip()


def make_few_shot_prompt(task, examples, constraints=None):
    """
    Few-shot: includes multiple examples.
    examples: list of (q, a)
    """
    c = f"\nConstraints: {constraints}" if constraints else ""
    ex_block = []
    for idx, (q, a) in enumerate(examples, 1):
        ex_block.append(f"Example {idx}:\nQuestion: {q}\nAnswer:\n{a}\n")
    ex_text = "\n".join(ex_block).strip()

    return f"""
Task: {task}{c}

{ex_text}

Now answer this:
Question: {task}
""".strip()



In [36]:
TOPIC_TASK = "Explain the concept of LLMs in healthcare."

COMMON_CONSTRAINTS = (
    "Plain text only. No Markdown. No stars. "
    "Use headings with colon. "
    "Follow: Plan, Main Answer, Example (if useful), Summary."
)


In [38]:
import os
from openai import OpenAI

# Groq client (reads from Colab Secrets)
client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)

def call_ai(
    user_prompt,
    model="llama-3.1-8b-instant",
    temperature=0.25,
    max_tokens=600
):
    messages = [
        {"role": "system", "content": ROOT_PROMPT},
        {"role": "user", "content": user_prompt}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )

    text = response.choices[0].message.content.strip()

    # Safety cleanup (no stars / markdown)
    text = text.replace("**", "").replace("*", "").replace("_", "")

    return text


In [39]:
prompt_cot = make_chain_of_thought_practice_prompt(TOPIC_TASK)

print("PROMPT USED:\n", prompt_cot, "\n")
print("MODEL OUTPUT:\n", call_ai(prompt_cot))


PROMPT USED:
 Role: Structured Academic Tutor
Task: Explain the concept of LLMs in healthcare.

Use this exact reasoning outline (plain text only):
Plan:
Key assumptions:
Steps:
Result:
Quick check:

Now produce the final output using that outline. 

MODEL OUTPUT:
 Plan:
1. Key assumptions: 
   - LLMs are a type of AI model.
   - LLMs are being increasingly used in various industries, including healthcare.
   - The primary goal of using LLMs in healthcare is to improve patient outcomes and care efficiency.

2. Steps:
   - Identify the key applications of LLMs in healthcare, such as clinical decision support, patient data analysis, and medical research.
   - Explain the benefits of using LLMs in healthcare, including improved accuracy, efficiency, and patient outcomes.
   - Discuss the challenges and limitations of using LLMs in healthcare, such as data quality, bias, and regulatory compliance.

3. Result:
   - A comprehensive understanding of the concept of LLMs in healthcare, includin

In [40]:
tab_prompt = make_tabular_prompt(
    topic="LLMs in healthcare",
    columns=["Category", "Description", "Example"],
    rows_hint="Include rows for definition, applications, benefits, limitations, safety"
)
print("PROMPT USED:\n", tab_prompt, "\n")
print("MODEL OUTPUT:\n", call_ai(tab_prompt))


PROMPT USED:
 Task: Create a plain-text table (NOT Markdown).
Topic: LLMs in healthcare

Rules:
- Output must be plain text only.
- Table must use columns: Category, Description, Example.
- Use a simple delimiter like: | between columns.
- One row per line.
- Keep each cell concise.
Row guidance: Include rows for definition, applications, benefits, limitations, safety

Return the table only. 

MODEL OUTPUT:
 Category|Description|Example
1.|Definition|Large Language Models (LLMs) are AI systems that process and generate human-like language.
2.|Applications|Medical diagnosis, patient education, clinical documentation, and medical research.
3.|Benefits|Improved accuracy, efficiency, and patient outcomes; enhanced clinician productivity.
4.|Limitations|Data bias, lack of transparency, and high computational requirements.
5.|Safety|Ensuring patient data protection, preventing misdiagnosis, and mitigating potential harm.


In [41]:
blanks = [
    ("Definition", "What LLMs are in one sentence"),
    ("Key healthcare use", "One major healthcare use case"),
    ("Two benefits", "Write two benefits"),
    ("Two risks", "Write two risks"),
    ("Example", "One realistic application example")
]
fib_prompt = make_fill_in_blank_prompt("LLMs in healthcare", blanks)
print("PROMPT USED:\n", fib_prompt, "\n")
print("MODEL OUTPUT:\n", call_ai(fib_prompt))


PROMPT USED:
 Topic: LLMs in healthcare
Fill in the blanks accurately:
1. Definition: ________ (What LLMs are in one sentence)
2. Key healthcare use: ________ (One major healthcare use case)
3. Two benefits: ________ (Write two benefits)
4. Two risks: ________ (Write two risks)
5. Example: ________ (One realistic application example)
Rules: plain text only. Do not add extra sections. 

MODEL OUTPUT:
 1. Definition: Large language models (LLMs) are artificial intelligence systems that can process, analyze, and generate human-like language to perform a wide range of tasks, from answering questions to completing complex writing and translation tasks.

2. Key healthcare use: Clinical decision support systems, where LLMs can analyze vast amounts of medical literature and patient data to provide healthcare professionals with accurate and up-to-date information to inform diagnosis and treatment decisions.

3. Two benefits: 
1. Improved accuracy in medical diagnosis and treatment through the a

In [42]:
rgc_prompt = make_rgc_prompt(
    role="You are The Structured Academic Tutor.",
    goal="Explain LLMs in healthcare for a second-year undergraduate audience.",
    constraints_must=[
        "Definition of LLMs",
        "At least 4 applications in healthcare",
        "At least 3 benefits",
        "At least 3 limitations/risks",
        "One example use case"
    ],
    constraints_avoid=[
        "Medical diagnosis or personalized treatment advice",
        "Unverifiable claims",
        "Markdown or decorative symbols"
    ],
    format_rules=[
        "Output sections: Plan:, Main Answer:, Example:, Summary:",
        "Plain text only; headings must use colon"
    ],
    length="250 to 350 words"
)
final_rgc_prompt = f"{rgc_prompt}\n\nNow produce the answer."
print("PROMPT USED:\n", final_rgc_prompt, "\n")
print("MODEL OUTPUT:\n", call_ai(final_rgc_prompt))


PROMPT USED:
 Role:
You are The Structured Academic Tutor.

Goal:
Explain LLMs in healthcare for a second-year undergraduate audience.

Constraints:
Must include:
- Definition of LLMs
- At least 4 applications in healthcare
- At least 3 benefits
- At least 3 limitations/risks
- One example use case

Must avoid:
- Medical diagnosis or personalized treatment advice
- Unverifiable claims
- Markdown or decorative symbols

Format rules:
- Output sections: Plan:, Main Answer:, Example:, Summary:
- Plain text only; headings must use colon
Length: 250 to 350 words

Now produce the answer. 

MODEL OUTPUT:
 Plan:
1. Define LLMs and their relevance to healthcare
2. Discuss applications of LLMs in healthcare
3. Outline benefits and limitations of LLMs in healthcare
4. Provide an example use case
5. Summarize key points

Refined Question:
What are Large Language Models and how are they being used in healthcare?

Verifier Checks:
1. Verify definition of LLMs is accurate
2. Ensure at least four disti

In [43]:
zero_prompt = make_zero_shot_prompt(TOPIC_TASK, constraints=COMMON_CONSTRAINTS)
print("ZERO-SHOT PROMPT:\n", zero_prompt, "\n")
print("ZERO-SHOT OUTPUT:\n", call_ai(zero_prompt))


ZERO-SHOT PROMPT:
 Task: Explain the concept of LLMs in healthcare.
Constraints: Plain text only. No Markdown. No stars. Use headings with colon. Follow: Plan, Main Answer, Example (if useful), Summary.
Return the answer now. 

ZERO-SHOT OUTPUT:
 Plan:
1. Define what LLMs are and their core capabilities.
2. Explain how LLMs are applied in healthcare.
3. Discuss the benefits and challenges of using LLMs in healthcare.

Refined Question:
What are Large Language Models and how are they being used in the healthcare industry?

Verifier Checks:
1. Check if the definition of LLMs is accurate and comprehensive.
2. Verify if the explanation of LLMs in healthcare is clear and concise.
3. Ensure that the benefits and challenges of using LLMs in healthcare are accurately stated.
4. Check if the answer includes a relevant example.
5. Review the summary for completeness and accuracy.

Main Answer:
Large Language Models (LLMs) are a type of artificial intelligence (AI) that are trained on vast amount

In [44]:
example_q = "Explain machine learning in finance."
example_a = (
    "Plan:\n"
    "1. Define machine learning in finance.\n"
    "2. Describe key applications.\n"
    "3. Discuss benefits and risks.\n"
    "\n"
    "Main Answer:\n"
    "Machine learning in finance refers to using algorithms that learn patterns from data to support tasks such as fraud detection, credit scoring, and trading analytics.\n"
    "It is valuable because financial datasets are large and change over time, and learned models can adapt to these patterns.\n"
    "\n"
    "Example:\n"
    "A bank may detect fraud by training a model on past transactions and flagging unusual spending behavior.\n"
    "\n"
    "Summary:\n"
    "Machine learning improves speed and accuracy for several financial tasks, but requires careful monitoring for bias, drift, and regulatory compliance."
)

one_prompt = make_one_shot_prompt(TOPIC_TASK, example_q, example_a, constraints=COMMON_CONSTRAINTS)
print("ONE-SHOT PROMPT:\n", one_prompt, "\n")
print("ONE-SHOT OUTPUT:\n", call_ai(one_prompt))


ONE-SHOT PROMPT:
 Task: Explain the concept of LLMs in healthcare.
Constraints: Plain text only. No Markdown. No stars. Use headings with colon. Follow: Plan, Main Answer, Example (if useful), Summary.

Example:
Question: Explain machine learning in finance.
Answer:
Plan:
1. Define machine learning in finance.
2. Describe key applications.
3. Discuss benefits and risks.

Main Answer:
Machine learning in finance refers to using algorithms that learn patterns from data to support tasks such as fraud detection, credit scoring, and trading analytics.
It is valuable because financial datasets are large and change over time, and learned models can adapt to these patterns.

Example:
A bank may detect fraud by training a model on past transactions and flagging unusual spending behavior.

Summary:
Machine learning improves speed and accuracy for several financial tasks, but requires careful monitoring for bias, drift, and regulatory compliance.

Now answer this:
Question: Explain the concept of

In [45]:
few_examples = [
    (
        "Explain cloud computing.",
        "Plan:\n1. Define cloud computing.\n2. Explain service models.\n3. Describe benefits and risks.\n\nMain Answer:\nCloud computing is the delivery of computing resources over the internet, allowing users to access storage, compute, and software on demand.\n\nExample:\nUsing online file storage to share documents across devices.\n\nSummary:\nCloud computing improves flexibility and scalability, but requires attention to security and vendor dependence."
    ),
    (
        "Explain computer networks.",
        "Plan:\n1. Define a network.\n2. Identify core components.\n3. Explain why protocols matter.\n\nMain Answer:\nA computer network connects devices so they can share data and resources using communication rules called protocols.\n\nExample:\nA home Wi-Fi network connecting phones and laptops to the internet.\n\nSummary:\nNetworks enable sharing and communication, but require correct configuration and security controls."
    )
]

few_prompt = make_few_shot_prompt(TOPIC_TASK, few_examples, constraints=COMMON_CONSTRAINTS)
print("FEW-SHOT PROMPT:\n", few_prompt, "\n")
print("FEW-SHOT OUTPUT:\n", call_ai(few_prompt))


FEW-SHOT PROMPT:
 Task: Explain the concept of LLMs in healthcare.
Constraints: Plain text only. No Markdown. No stars. Use headings with colon. Follow: Plan, Main Answer, Example (if useful), Summary.

Example 1:
Question: Explain cloud computing.
Answer:
Plan:
1. Define cloud computing.
2. Explain service models.
3. Describe benefits and risks.

Main Answer:
Cloud computing is the delivery of computing resources over the internet, allowing users to access storage, compute, and software on demand.

Example:
Using online file storage to share documents across devices.

Summary:
Cloud computing improves flexibility and scalability, but requires attention to security and vendor dependence.

Example 2:
Question: Explain computer networks.
Answer:
Plan:
1. Define a network.
2. Identify core components.
3. Explain why protocols matter.

Main Answer:
A computer network connects devices so they can share data and resources using communication rules called protocols.

Example:
A home Wi-Fi net

In [46]:
def run_interactive():
    print("Modes: cot, table, fib, rgc, zero, one, few")
    mode = input("Choose mode: ").strip().lower()

    while True:
        q = input("\nEnter your question (or type exit): ").strip()
        if q.lower() in ["exit", "quit"]:
            print("Session ended.")
            break

        if mode == "cot":
            prompt = make_chain_of_thought_practice_prompt(q)
        elif mode == "table":
            prompt = make_tabular_prompt(q, ["Aspect", "Details", "Example"])
        elif mode == "fib":
            prompt = make_fill_in_blank_prompt(q, [("Definition", "one sentence"), ("Example", "one example")])
        elif mode == "rgc":
            prompt = make_rgc_prompt(
                role="You are The Structured Academic Tutor.",
                goal=f"Answer: {q}",
                constraints_must=["Clear explanation", "At least 1 example if useful"],
                constraints_avoid=["Markdown", "Decorative formatting"],
                format_rules=["Sections: Plan:, Main Answer:, Example:, Summary:"]
            ) + "\n\nNow produce the answer."
        elif mode == "zero":
            prompt = make_zero_shot_prompt(q, constraints=COMMON_CONSTRAINTS)
        elif mode == "one":
            prompt = make_one_shot_prompt(q, example_q, example_a, constraints=COMMON_CONSTRAINTS)
        elif mode == "few":
            prompt = make_few_shot_prompt(q, few_examples, constraints=COMMON_CONSTRAINTS)
        else:
            prompt = q  # fallback

        print("\nAI OUTPUT:\n")
        print(call_ai(prompt))

run_interactive()


Modes: cot, table, fib, rgc, zero, one, few
Choose mode: table

Enter your question (or type exit): states in india which are in the south

AI OUTPUT:

1. Aspect | Details | Example
2. State | Andhra Pradesh | Capital: Amaravati
3. State | Telangana | Capital: Hyderabad
4. State | Karnataka | Capital: Bengaluru
5. State | Tamil Nadu | Capital: Chennai
6. State | Kerala | Capital: Thiruvananthapuram
7. State | Puducherry | Capital: Puducherry

Enter your question (or type exit): exit
Session ended.
